## Introduction to BGP Analysis using Batfish

Network engineers routinely need to validate BGP configuration and session status in the network. The most common way to accomplish this is by connecting to multiple network devices and executing a series of `show ip bgp` commands. This distributed debugging is highly complex even in a moderately-sized network. Batfish makes BGP analysis extremely simple by providing an easy-to-query, centralized view of routing tables in the network. 

In this notebook, we will show how you can extract BGP configuration and session status information with Batfish.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-bgp-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

In [2]:
# Initialize a network and snapshot
NETWORK_NAME = "bgp_network"
SNAPSHOT_NAME = "bgp_snapshot"

SNAPSHOT_PATH = "networks/example-bgp"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'bgp_snapshot'

now tell them about the network and the issue: Interface utilization on core routers is badly distributed

Traffic from dept disproportionately hits core2 instead of core1 *(reason: dist2 doesn't have a route to core1)*

## BGP process configuration
First, let's ensure that BGP is configured on the routers in question. The `BgpProcessConfiguration` question lists BGP processes configured on a given set of nodes, or on all nodes if no parameters are given.

In [3]:
bfq.bgpProcessConfiguration(nodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Router_ID,Multipath_EBGP,Multipath_IBGP,Tie_Breaker,Neighbors,Route_Reflector,Multipath_Match_Mode
0,as2dist2,default,2.1.3.2,True,True,ARRIVAL_ORDER,"['2.1.2.1/32', '2.1.2.2/32', '2.34.0.0/16']",False,EXACT_PATH
1,as2core1,default,2.1.2.1,True,True,ARRIVAL_ORDER,"['2.1.1.1/32', '2.1.1.2/32', '2.1.3.1/32', '2.1.3.2/32']",True,EXACT_PATH


The results of `BgpProcessConfiguration` show that `as2core1` and `as2dist2` do have BGP configured. Let's look at their configured peers.

## BGP peer configuration
Next we can look for any issues with the individual BGP peers configured on `as2core1` and `as2dist2`. The `BgpPeerConfiguration` question shows all BGP peers configured on a given set of nodes, or on all nodes if no parameters are given.

In [4]:
# BgpPeerConfiguration shows all configured BGP peers
bfq.bgpPeerConfiguration(nodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Local_AS,Local_IP,Remote_AS,Remote_IP,Route_Reflector_Client,Cluster_ID,Peer_Group,Import_Policy,Export_Policy,Send_Community,Is_Passive
0,as2dist2,default,2,2.1.3.2,2,2.1.2.2,False,None,as2,[],[],True,False
1,as2core1,default,2,2.1.2.1,2,2.1.1.1,True,2.1.2.1,as2,[],[],True,False
2,as2dist2,default,2,2.1.3.2,2,2.1.2.1,False,None,as2,[],[],True,False
3,as2core1,default,2,2.1.2.1,2,2.1.3.1,True,2.1.2.1,as2,[],[],True,False
4,as2core1,default,2,2.1.2.1,2,2.1.1.2,True,2.1.2.1,as2,[],[],True,False
5,as2core1,default,2,2.1.2.1,2,2.1.3.2,True,2.1.2.1,as2,[],[],True,False
6,as2dist2,default,2,AUTO/NONE(-1l),[65001],2.34.0.0/16,False,None,dept,['dept_to_as2dist'],['as2dist_to_dept'],True,True


Rows 2 and 5 appear to be correctly configured to peer with each other.

## BGP session status
Next, let's see if the BGP session between `as2core1` and `as2dist2` will get established. The `BgpSessionStatus` question shows which compatible BGP peers establish sessions. Unlike the previous questions, this question relies on Batfish's generated data plane to provide information that would normally only be accessible by querying the live network, so it will take longer to run the first time while the data plane gets built.

In [5]:
bfq.bgpSessionStatus(nodes='as2core1|as2dist2', remoteNodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Established_Status
0,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist2,2.1.3.2,IBGP,NOT_ESTABLISHED
1,as2dist2,default,2,as2dist2:Loopback0,2.1.3.2,2,as2core1,2.1.2.1,IBGP,NOT_ESTABLISHED


Now we can see that the BGP session did *not* get established between the BGP peers on `as2core1` and `as2dist2`, even though they seemed compatible based on the results of `BgpPeerConfiguration`.

## BGP session compatibility
Did we miss something when we looked at the results of `BgpPeerConfiguration`? The `BgpSessionCompatibility` question shows the compatibility of BGP sessions, which should confirm whether `as2core1` and `as2dist2` are compatible. Sessions can have various compatibility statuses. A correctly configured peer will have one of the following statuses:

- `UNIQUE_MATCH`: The peer is active and has exactly one compatible remote peer.
- `DYNAMIC_MATCH`: The peer is passive and has at least one compatible remote peer.
- `UNKNOWN_REMOTE`: The peer is active, but its remote IP is not associated with a known interface. Most likely, this peer is intended to establish a session with an external device, but if not, it is improperly configured.
- `NO_MATCH_FOUND`: The peer is passive, but no active peers in the network are compatible with it. Most likely, this peer is expecting sessions to be initiated by external devices, but if not, it is improperly configured.

Other statuses may indicate basic configuration issues, such as `NO_LOCAL_IP` or `NO_REMOTE_AS`, as well as more subtle issues:

- `INVALID_LOCAL_IP`: The peer is configured with a local IP that isn't associated with a known interface.
- `HALF_OPEN`: The peer is active and its remote IP is recognized, but no compatible peer exists on the remote interface.
- `MULTIPLE_REMOTES`: The peer is active with multiple remote peers configured compatibly.

If `as2core1` and `as2dist2` are compatibly configured, we should see `UNIQUE_MATCH` for their session.

In [6]:
bfq.bgpSessionCompatibility(nodes='as2core1|as2dist2', remoteNodes='as2core1|as2dist2').answer().frame()

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
0,as2core1,default,2,as2core1:Loopback0,2.1.2.1,2,as2dist2,2.1.3.2,IBGP,UNIQUE_MATCH
1,as2dist2,default,2,as2dist2:Loopback0,2.1.3.2,2,as2core1,2.1.2.1,IBGP,UNIQUE_MATCH


Looking at the results for the session between `as2core1` and `as2dist2`, we are now confident that the BGP peers are correctly configured. There must be some other problem preventing the session from being established.

While we're looking at `BgpSessionCompatibility`, let's check the rest of the network for any BGP sessions that may not be compatible. We will limit the output to peers whose configured status is *not* `UNIQUE_MATCH` or `DYNAMIC_MATCH`.

In [7]:
all_session_compatibility = bfq.bgpSessionCompatibility().answer().frame()
all_session_compatibility[
    (all_session_compatibility['Configured_Status'] != 'UNIQUE_MATCH')
    & (all_session_compatibility['Configured_Status'] != 'DYNAMIC_MATCH')]

,Node,VRF,Local_AS,Local_Interface,Local_IP,Remote_AS,Remote_Node,Remote_IP,Session_Type,Configured_Status
1,as1border1,default,1,None,None,666,None,3.2.2.2,EBGP_SINGLEHOP,NO_LOCAL_IP
2,as1border1,default,1,None,None,555,None,5.6.7.8,EBGP_SINGLEHOP,NO_LOCAL_IP
6,as1border2,default,1,as1border2:GigabitEthernet2/0,10.14.22.1,4,None,10.14.22.4,EBGP_SINGLEHOP,UNKNOWN_REMOTE
13,as2border2,default,2,None,None,2,None,2.1.2.2,IBGP,LOCAL_IP_UNKNOWN_STATICALLY
20,as2core2,default,2,as2core2:Loopback0,2.1.2.2,2,None,2.1.1.2,IBGP,HALF_OPEN


Of these results, the first two have no local IP configured, so will be unable to establish sessions: almost certainly an oversight that should be fixed. The third has a remote IP outside of the network Batfish is modeling, so it may be able to establish a session in the live network, but Batfish doesn't have enough information to be sure.

### TODO explain last two results

These three results do not offer an explanation for the problem at hand, so let's keep looking.

## Check routes
Now we know the BGP peers on `as2core1` and `as2dist2` are compatible, but the session is not established. Let's make sure both devices have a route to the interface of each other's BGP peers. Looking at any of the last few questions concerning these two nodes, we can see the IPs of these interfaces:

- BGP peer on `as2core1` has local IP `2.1.2.1`
- BGP peer on `as2dist2` has local IP `2.1.3.2`

To see if the devices have the necessary routes to each other, we can search the output of the `Routes` question.

In [22]:
# See if as2core1 has a route to 2.1.3.2 (the local IP for as2dist2's BGP peer)
routes = bfq.routes().answer().frame()
routes[(routes['Node'] == 'as2core1') & (routes['Network'] == '2.1.3.2/32')]

,Node,VRF,Network,Protocol,Next_Hop_IP,Next_Hop,Admin_Distance,Metric,Tag
220,as2core1,default,2.1.3.2/32,ospf,2.23.12.3,as2dist2,110,2,None


In [23]:
# See if as2dist2 has a route to 2.1.2.1 (the local IP for as2core1's BGP peer)
routes[(routes['Node'] == 'as2dist2') & (routes['Network'] == '2.1.2.1/32')]

,Node,VRF,Network,Protocol,Next_Hop_IP,Next_Hop,Admin_Distance,Metric,Tag


The mystery is solved! Node `as2dist2` has no route for its BGP peer's remote IP, so even though the BGP peer was configured correctly, the session could not be established.

***
## Get involved with the Batfish community

Join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 